In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
import csv
import shutil
import os
import time
import re
import nltk
nltk.download('wordnet')
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import numpy as np

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pickle
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.cluster import MiniBatchKMeans

In [3]:
from collections import Counter
from random import seed
from random import randrange
from tabulate import tabulate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [4]:
#https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt
import requests
base_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/"
file_name = "amazon_reviews_us_Wireless_v1_00"
r = requests.get(base_url+file_name+".tsv.gz", allow_redirects=True)
open(file_name+".tsv.gz", 'wb').write(r.content)

with gzip.open(file_name+".tsv.gz", 'rb') as f_in:
    with open(file_name+".tsv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

os.remove(file_name+".tsv"+".gz")
chunk = pd.read_csv(file_name+".tsv", sep='\t',error_bad_lines=False,chunksize=1000000,low_memory=False)
df = pd.concat(chunk)
#Removing null values
df = df.dropna()
os.remove(file_name+".tsv")
print(len(df))
print(df.head())
print("Before cleanning reviews")
print(df["review_body"].head(5))
def clean_dataset(X):
  review = X
  stemmer = WordNetLemmatizer()

  # Remove all the special characters
  review = re.sub(r'\W', ' ', review)
  
  # remove all single characters
  review = re.sub(r'\s+[a-zA-Z]\s+', ' ', review)
  
  # Remove single characters from the start
  review = re.sub(r'\^[a-zA-Z]\s+', ' ', review) 
  
  # Substituting multiple spaces with single space
  review = re.sub(r'\s+', ' ', review, flags=re.I)
  
  # Removing prefixed 'b'
  review = re.sub(r'^b\s+', '', review)
  
  # Converting to Lowercase
  review = review.lower()
  
  # Lemmatization
  review = review.split()
  review = [stemmer.lemmatize(word) for word in review]
  review = ' '.join(review)

  #Removing all  stopwords.
  review = remove_stopwords(review)
  return review

start_time = time.time()
def combined_features(row):
    return row['product_title'] + ' '+row['review_headline']+' '+ row['review_body']

X = df.apply(combined_features, axis=1)

X = df["review_body"].apply(lambda x: clean_dataset(str(x)))
print(f"Total time to clean reviews: {time.time()-start_time}")
print("After cleanning reviews")
print(X.head(5))
y = df["star_rating"]
from google.colab import drive
drive.mount('/content/drive')
datasetPath = "/content/drive/My Drive/Natural Language Processing/Project/Cleaned Dataset/"
#Saving to CSV File
X.to_csv(datasetPath+'X_cleaned.csv', index=False) 
y.to_csv(datasetPath+'y_cleaned.csv', index=False) 

In [5]:
from google.colab import drive
drive.mount('/content/drive')
datasetPath = "/content/drive/My Drive/Natural Language Processing/Project/Cleaned Dataset/"
start_time = time.time()
X = pd.read_csv(datasetPath+'X_cleaned.csv')
y = pd.read_csv(datasetPath+'y_cleaned.csv')
print(f"Total time to load reviews: {time.time()-start_time}")

#Removing null values
result = pd.concat([X,y], axis=1).dropna()
result.head(10)
X = result["review_body"]
y = result["star_rating"]

#Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = np.array(X_train).reshape(X_train.shape[0],)
X_test = np.array(X_test).reshape(X_test.shape[0],)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total time to load reviews: 27.463109016418457
(6013513,)
(6013513,)
(2961880,)
(2961880,)


In [6]:
# Tfidf vectorizer:
vectorizer = TfidfVectorizer(max_df=0.7, max_features=1000,
                             min_df=5, stop_words='english',
                             use_idf=True)

# Build the tfidf vectorizer from the training data ("fit"), and apply it 
# ("transform").
X_train_tfidf = vectorizer.fit_transform(X_train)

print(f"  Actual number of tfidf features: {X_train_tfidf.get_shape()}")

print("\nPerforming dimensionality reduction using LSA")
t0 = time.time()

# Project the tfidf vectors onto the first N principal components.
# Though this is significantly fewer features than the original tfidf vector,
# they are stronger features, and the accuracy is higher.
svd = TruncatedSVD(100)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
X_train_lsa = lsa.fit_transform(X_train_tfidf)

print("  done in %.3fsec" % (time.time() - t0))

# explained_variance = svd.explained_variance_ratio_.sum()
# print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

# Now apply the transformations to the test data as well.
X_test_tfidf = vectorizer.transform(X_test)
X_test_lsa = lsa.transform(X_test_tfidf)

  Actual number of tfidf features: (6013513, 1000)

Performing dimensionality reduction using LSA
  done in 373.250sec


In [7]:
#scaling vector between 0 - 1 
scaler = MinMaxScaler()
scaler.fit(X_train_lsa)
X_train_lsa_scaled =scaler.transform(X_train_lsa)

scaler = MinMaxScaler()
scaler.fit(X_test_lsa)
X_test_lsa_scaled =scaler.transform(X_test_lsa)

#Convert all rating to type int
y_train = np.array(y_train).astype(int)
y_test = np.array(y_test).astype(int)


In [8]:
y_test_old = y_test 
y_train_old = y_train

In [9]:
y_test = y_test_old 
y_train = y_train_old
#Using only 3 ratings:
def rating(r):
  if r == 1:
    return 0;
  if r == 2:
    return 1;
  if r == 3:
    return 2;
  if r == 4:
    return 3;
  if r == 5:
    return 4;
ratings = lambda t: rating(t)
vfunc = np.vectorize(ratings)
y_train = vfunc(y_train)
y_test = vfunc(y_test)

print(np.unique(y_train))

[0 1 2 3 4]


In [10]:
target_names = []
labels = np.unique(y_test)
for label in labels:
  target_names.append('Star Rating '+str(label))
print(f"Target Names: {[target_names]}")

Target Names: [['Star Rating 0', 'Star Rating 1', 'Star Rating 2', 'Star Rating 3', 'Star Rating 4']]


In [11]:
from sklearn.linear_model import SGDClassifier
n_batches = 10
current_batch = 0
increment = (int(len(X_train_lsa_scaled)/n_batches))
clf = SGDClassifier(loss="hinge", penalty="l2")

print(f"Classifying using SVM")
start_time0 = time.time()
for i in range(n_batches):
  start_time = time.time()
  next_batch = current_batch + increment
  clf.partial_fit(X_train_lsa_scaled[current_batch:next_batch], y_train[current_batch:next_batch], classes=labels)
  current_batch = next_batch
  print(f"Batch {i} : {current_batch} of {len(X_train_lsa_scaled)} : {time.time() - start_time} seconds ")
print(f"Total training time: {time.time() - start_time0} ")
start_time = time.time()
svm_accuracy = accuracy_score(y_test, clf.predict(X_test_lsa_scaled))
print(f"Test dataset accuracy for classifier is {svm_accuracy} ")
print(classification_report(y_test, clf.predict(X_test_lsa_scaled), target_names=target_names))
print(f"Total time to test: {time.time()-start_time}")

Classifying using SVM
Batch 0 : 601351 of 6013513 : 2.013427495956421 seconds 
Batch 1 : 1202702 of 6013513 : 1.9768202304840088 seconds 
Batch 2 : 1804053 of 6013513 : 1.975494384765625 seconds 
Batch 3 : 2405404 of 6013513 : 2.423121452331543 seconds 
Batch 4 : 3006755 of 6013513 : 1.970905065536499 seconds 
Batch 5 : 3608106 of 6013513 : 2.004537582397461 seconds 
Batch 6 : 4209457 of 6013513 : 1.9648942947387695 seconds 
Batch 7 : 4810808 of 6013513 : 2.000108242034912 seconds 
Batch 8 : 5412159 of 6013513 : 1.9616999626159668 seconds 
Batch 9 : 6013510 of 6013513 : 1.9842031002044678 seconds 
Total training time: 20.27700448036194 
Test dataset accuracy for classifier is 0.6064688643699272 
               precision    recall  f1-score   support

Star Rating 0       0.54      0.58      0.56    415969
Star Rating 1       0.24      0.00      0.00    196705
Star Rating 2       0.39      0.04      0.07    268689
Star Rating 3       0.32      0.04      0.07    494257
Star Rating 4      

In [12]:
start_time = time.time()
print(f"Classifying using multinomialNB")
clf = MultinomialNB()
clf.fit(X_train_lsa_scaled, y_train)
print(f"Total time to train: {time.time()-start_time}")
start_time = time.time()
nb_accuracy = accuracy_score(y_test, clf.predict(X_test_lsa_scaled))
print(f"Test dataset accuracy is {nb_accuracy} ")
print(classification_report(y_test, clf.predict(X_test_lsa_scaled), target_names=target_names))
print(f"Total time to test: {time.time()-start_time}")

Classifying using multinomialNB
Total time to train: 3.4640023708343506
Test dataset accuracy is 0.5355584966305185 


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

Star Rating 0       0.00      0.00      0.00    415969
Star Rating 1       0.00      0.00      0.00    196705
Star Rating 2       0.00      0.00      0.00    268689
Star Rating 3       0.00      0.00      0.00    494257
Star Rating 4       0.54      1.00      0.70   1586260

     accuracy                           0.54   2961880
    macro avg       0.11      0.20      0.14   2961880
 weighted avg       0.29      0.54      0.37   2961880

Total time to test: 7.6708197593688965


In [13]:
n_batches = 1
current_batch = 0
increment = (int(len(X_train_lsa_scaled)/n_batches))
clf = MiniBatchKMeans(n_clusters=len(labels),random_state=0,batch_size=n_batches)

print(f"Classifying using KNN")
start_time0 = time.time()
for i in range(n_batches):
  start_time = time.time()
  next_batch = current_batch + increment
  clf.partial_fit(X_train_lsa_scaled[current_batch:next_batch])
  current_batch = next_batch
  print(f"Batch {i} : {current_batch} of {len(X_train_lsa_scaled)} : {time.time() - start_time} seconds ")
print(f"Total training time: {time.time() - start_time0} ")
start_time = time.time()
knn_accuracy = accuracy_score(y_test, clf.predict(X_test_lsa_scaled))
print(f"Test dataset accuracy for classifier is {svm_accuracy} ")
print(classification_report(y_test, clf.predict(X_test_lsa_scaled), target_names=target_names))
print(f"Total time to test: {time.time()-start_time}")

Classifying using KNN
Batch 0 : 6013513 of 6013513 : 18.13900923728943 seconds 
Total training time: 18.139588594436646 
Test dataset accuracy for classifier is 0.6064688643699272 
               precision    recall  f1-score   support

Star Rating 0       0.24      0.23      0.24    415969
Star Rating 1       0.06      0.16      0.09    196705
Star Rating 2       0.08      0.23      0.12    268689
Star Rating 3       0.12      0.13      0.12    494257
Star Rating 4       0.48      0.25      0.32   1586260

     accuracy                           0.22   2961880
    macro avg       0.20      0.20      0.18   2961880
 weighted avg       0.32      0.22      0.24   2961880

Total time to test: 2345.685484647751
